In [2]:
!pip install transformers evaluate scikit-learn scipy pandas

In [2]:
import torch
import copy
import pandas as pd 
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DebertaV2Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# PRETRAIN_MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
PRETRAIN_MODEL = "microsoft/deberta-v3-base"
BEST_TRIAL = 11


#### Load data

In [3]:
train_df = pd.read_csv("data/split_train_data.csv").dropna(subset=["Label"]).reset_index(drop=True)
eval_df = pd.read_csv("data/split_eval_data.csv").dropna(subset=["Label"]).reset_index(drop=True)

#### Load Best Model

In [4]:
CKPT_PT = f"models/training/deberta_best_model_trial_{BEST_TRIAL}.pt"
tokenizer = AutoTokenizer.from_pretrained(PRETRAIN_MODEL, use_fast=True)
best_model = AutoModelForSequenceClassification.from_pretrained(PRETRAIN_MODEL, num_labels=5)

state = torch.load(CKPT_PT, map_location="cpu")
best_model.load_state_dict(state) 
best_model.eval()

from transformers import DebertaV2Tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=True)

baseline_num_params = sum(p.numel() for p in best_model.parameters())

/home/idok/git/deep-tweet-covid/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 1. QUANTIZATION DEMO

In [ ]:
from torch.ao.quantization import quantize_dynamic
from torch import nn

# Weight-only quantization to FP16
quantized_model = quantize_dynamic(
    best_model.cpu().eval(),
    {nn.Linear},
    dtype=torch.float16,   # FP16 instead of int8
    inplace=False
)

quantized_num_params = sum(p.numel() for p in quantized_model.parameters())
print("Original model params:", baseline_num_params)
print("Quantized model params:", quantized_num_params)

Original model params: 184425989
Quantized model params: 98813952


#### 2. Pruning

In [ ]:
from torch.nn.utils import prune

# Only prune FFN dense layers, not attention or classifier
def collect_ffn_linears(model):
    pairs = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            if ".intermediate.dense" in name or ".output.dense" in name:
                pairs.append((module, "weight"))
    return pairs

model_to_prune = copy.deepcopy(best_model)
parameters_to_prune = collect_ffn_linears(model_to_prune)

# Global L1 pruning across FFN weights (40%)
prune.global_unstructured(parameters_to_prune, prune.L1Unstructured, amount=0.4)

# Remove masks (make pruning permanent)
for m, _ in parameters_to_prune:
    prune.remove(m, "weight")


print("Pruning applied: 40% of weights zeroed out in Linear layers.")
pruned_num_params = sum((p != 0).sum().item() for p in model_to_prune.parameters())
print("Remaining non-zero parameters:", pruned_num_params)


#### 3 Distillation

In [9]:
from torch.nn import functional as F
from transformers import Trainer, TrainingArguments

# Teacher = your best trained model
teacher = best_model.to(device)
teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)

# Student = smaller pre-trained model
student_name = "microsoft/deberta-v3-small"
student = AutoModelForSequenceClassification.from_pretrained(student_name, num_labels=5).to(device)

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs_student = model(**inputs)
        with torch.no_grad():
            #outputs_teacher = self.teacher(**inputs)
            outputs_teacher = self.teacher(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask", None)
            )

        labels = inputs["labels"].to(outputs_student.logits.device)
        labels = labels.long().view(-1)

        loss_ce = F.cross_entropy(outputs_student.logits, labels)
        loss_kl = F.kl_div(
            F.log_softmax(outputs_student.logits / self.temperature, dim=-1),
            F.softmax(outputs_teacher.logits / self.temperature, dim=-1),
            reduction="batchmean",
        ) * (self.temperature ** 2)

        loss = self.alpha * loss_ce + (1 - self.alpha) * loss_kl
        return (loss, outputs_student) if return_outputs else loss

# Wrap your datasets in torch Dataset -> DataLoader is handled by Trainer
from datasets import Dataset as HFDataset

train_hf = HFDataset.from_pandas(train_df.rename(columns={"Label": "labels"}))
eval_hf = HFDataset.from_pandas(eval_df.rename(columns={"Label": "labels"}))

def tokenize_fn(batch):
    texts = [str(x) for x in batch["OriginalTweet"]] 
    tokenized = tokenizer(texts, padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = [int(x) for x in batch["labels"]]
    return tokenized
    

train_tok = train_hf.map(tokenize_fn, batched=True)
eval_tok = eval_hf.map(tokenize_fn, batched=True)
train_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/28808 [00:00<?, ? examples/s]

Map:   0%|          | 0/12348 [00:00<?, ? examples/s]

In [10]:
args = TrainingArguments(
    output_dir="./distill_out",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    logging_steps=50,
    save_strategy="no",
)

trainer = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=args,
    train_dataset=train_tok,
    eval_dataset=eval_tok,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: yoyulia (yoyulia-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.210200,1.259324
2,0.763100,1.433853
3,0.479100,1.504478


TrainOutput(global_step=5403, training_loss=1.0525159482578206, metrics={'train_runtime': 1281.9362, 'train_samples_per_second': 67.417, 'train_steps_per_second': 4.215, 'total_flos': 2862345689229312.0, 'train_loss': 1.0525159482578206, 'epoch': 3.0})

In [11]:
print("\nDistillation complete. Student model trained.")
distilled_num_params = sum(p.numel() for p in best_model.parameters())
print("Student model size:", distilled_num_params)


Distillation complete. Student model trained.
Student model size: 184425989


In [23]:
import os
import torch

model_name = PRETRAIN_MODEL.split("/")[1]
output_model_path = f"models/{model_name}-full"

os.makedirs(output_model_path, exist_ok=True)

### Save compress models

In [15]:
# Baseline
best_model.save_pretrained(f"{output_model_path}/baseline")
tokenizer.save_pretrained(f"{output_model_path}/baseline")

# Quantized
torch.save(
    quantized_model.state_dict(),
    f"{output_model_path}/quantized.pt"
)
tokenizer.save_pretrained(f"{output_model_path}/quantized")  # same tokenizer

# Pruned
model_to_prune.save_pretrained(f"{output_model_path}/pruned")
tokenizer.save_pretrained(f"{output_model_path}/pruned")

# Distilled
student.save_pretrained(f"{output_model_path}/distilled")
tokenizer.save_pretrained(f"{output_model_path}/distilled")

('models/2-deberta-v3-base-full/distilled/tokenizer_config.json',
 'models/2-deberta-v3-base-full/distilled/special_tokens_map.json',
 'models/2-deberta-v3-base-full/distilled/spm.model',
 'models/2-deberta-v3-base-full/distilled/added_tokens.json')

### Load compress models (if needed)

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Load Baseline
# student = AutoModelForSequenceClassification.from_pretrained(f"{output_model_path}/baseline")
# distilled_tokenizer = AutoTokenizer.from_pretrained(f"{output_model_path}/baseline")

# # Load Quantized (CPU only!)
# quantized_tokenizer = AutoTokenizer.from_pretrained(f"{output_model_path}/quantized")
# quantized_model = AutoModelForSequenceClassification.from_pretrained(
#     PRETRAIN_MODEL, num_labels=5
# )
# quantized_model.load_state_dict(torch.load(f"{output_model_path}/quantized.pt"))
# quantized_model = quantize_dynamic(quantized_model, {torch.nn.Linear}, dtype=torch.qint8)

# # Load Pruned
# model_to_prune = AutoModelForSequenceClassification.from_pretrained(f"{output_model_path}/pruned")
# pruned_tokenizer = AutoTokenizer.from_pretrained(f"{output_model_path}/pruned")

# # Load Distilled
# student = AutoModelForSequenceClassification.from_pretrained(f"{output_model_path}/distilled")
# distilled_tokenizer = AutoTokenizer.from_pretrained(f"{output_model_path}/distilled")

### Compare the models

In [ ]:
import wandb

model_name = PRETRAIN_MODEL.split("/")[1]
wandb.init(
    project=f"model_compression-{model_name}",
    name=f"Baseline-Quantized-Pruned-Distilled-{model_name}",
    mode="online"  # use "offline" if no internet
)

import evaluate
from sklearn.metrics import roc_auc_score
from scipy.special import softmax
import numpy as np
import pandas as pd
from transformers import Trainer

# Define compute_metrics
metric_acc = evaluate.load("accuracy")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    precision = metric_precision.compute(predictions=predictions, references=labels, average="macro")
    recall = metric_recall.compute(predictions=predictions, references=labels, average="macro")
    f1 = metric_f1.compute(predictions=predictions, references=labels, average="macro")

    probs = softmax(logits, axis=1)
    try:
        auc = roc_auc_score(labels, probs, multi_class="ovr", average="macro")
    except ValueError:
        auc = float("nan")

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
        "auc": auc,
    }

# Create lightweight Trainer wrapper for evaluation
def evaluate_model(model, name, eval_dataset=eval_tok):
    use_cuda = torch.cuda.is_available() and name != "Quantized"
    model.to("cuda" if use_cuda else "cpu")
    model.eval()   # ensure eval mode
    
    args = TrainingArguments(
        output_dir="./tmp",
        per_device_eval_batch_size=16,
        no_cuda=not use_cuda,
        report_to="none",
    )
    
    trainer = Trainer(model=model, args=args, eval_dataset=eval_dataset, compute_metrics=compute_metrics)
    results = trainer.evaluate()
    
    dense_params = sum(p.numel() for p in model.parameters())
    nonzero_params = sum((p != 0).sum().item() for p in model.parameters())
    results.update({
        "params_dense": dense_params,
        "params_nonzero": nonzero_params,
        "model": name,
    })
    return results

# Evaluate all models
results = []
results.append(evaluate_model(model_to_prune, "Pruned"))
results.append(evaluate_model(student, "Distilled"))
results.append(evaluate_model(best_model, "Baseline"))
results.append(evaluate_model(quantized_model, "Quantized"))

# Put results in DataFrame
results_df = pd.DataFrame(results)
results_df = results_df[["model", "params_dense", "params_nonzero",
                         "eval_loss", "eval_accuracy", "eval_precision",
                         "eval_recall", "eval_f1", "eval_auc"]]


eval/loss,▁▆█
eval/runtime,▁█▂
eval/samples_per_second,█▁▇
eval/steps_per_second,█▁▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▂▂▃▄▆▃█▄▅▅▄▃▃▃▅▅▂▃▃▅▅█▁▃▂▅▄▃▅▅▃▂▂▃▁▁▂▆▂
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
train/loss,█▇▆▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
eval/loss,1.50448
eval/runtime,77.2553


/home/idok/git/deep-tweet-covid/.venv/lib/python3.10/site-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


RuntimeError: apply_dynamic is not implemented for this packed parameter type

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df[["model", "params_dense", "params_nonzero",
                         "eval_loss", "eval_accuracy", "eval_precision",
                         "eval_recall", "eval_f1", "eval_auc"]]


In [22]:
results_df

,model,params_dense,params_nonzero,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_auc
0,Pruned,184425989,158931990,1.397693,0.672174,0.734369,0.662508,0.682099,0.908969
1,Distilled,141898757,141889791,0.615705,0.854794,0.856643,0.865324,0.859554,0.976708
2,Baseline,184425989,184412367,1.475548,0.725219,0.732111,0.739021,0.735181,0.928169
3,Quantized,98813952,98800357,1.475523,0.725219,0.732074,0.739021,0.735162,0.928169


In [24]:
results_df.to_csv(f"{output_model_path}/comparison.csv")
results_df

,model,params_dense,params_nonzero,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_auc
0,Pruned,184425989,158931990,1.397693,0.672174,0.734369,0.662508,0.682099,0.908969
1,Distilled,141898757,141889791,0.615705,0.854794,0.856643,0.865324,0.859554,0.976708
2,Baseline,184425989,184412367,1.475548,0.725219,0.732111,0.739021,0.735181,0.928169
3,Quantized,98813952,98800357,1.475523,0.725219,0.732074,0.739021,0.735162,0.928169
